# Chapter 5 - Exercises

### 1. Write a recursive function to compute the factorial of a number.

In [10]:
def recursive_factorial(n):
    return 1 if n in (0, 1) else n * recursive_factorial(n - 1)


### 2. Write a recursive function to reverse a list.

In [3]:
def reverse_list(ls):
    if len(ls) in (0, 1):
        return ls 
    else:
        return [ls[-1]] + reverse_list(ls[1:-1]) + [ls[0]]
        

### 3. Modify the recursive tree program using one or all of the following ideas:

* ### Modify the thickness of the branches so that as the `branch_len` gets smaller, the line gets thinner.

* ### Modify the color of the branches so that as the `branch_len` gets very short it is colored like a leaf.

* ### Modify the angle used in turning the turtle so that at each branch point the angle is selected at random in some range. For example choose the angle between 15 and 45 degrees. Play around to see what looks good.

* ### Modify the `branch_len` recursively so that instead of always subtracting the same amount you subtract a random amount in some range.

### If you implement all of the above ideas you will have a very realistic looking tree.

We already answered that question in the relevant paragraph (link to the script (here)[./fractal.py]).

In [1]:
import turtle
import random

def tree(branch_len, min_step, max_step, t):
    t.color((0, 255 - branch_len if branch_len >= 0 else 255, 0))
    t.pensize(branch_len ** (1 / 2))
    a1 = random.randint(15, 45)
    a2 = random.randint(15, 45)
    step = random.randint(min_step, max_step)
    if branch_len >= min_step:
        t.forward(branch_len)
        t.right(a1)
        tree(branch_len - step, min_step, max_step, t)
        t.left(a1 + a2)
        tree(branch_len - step, min_step, max_step, t)
        t.right(a2)
        t.up()
        t.backward(branch_len)
        t.down()


### 5. Find or invent an algorithm for drawing a fractal mountain. Hint: One approach to this uses triangles again.